In [1]:
# Azure Notebook에서 Python 3.8 설정 후 실행


from azureml.core import Workspace
ws = Workspace.from_config()    # Workspace 설정
print('Workspace name : '+ws.name,'\n'
        'Azure region : '+ws.location,'\n'
        'Subscription ID : '+ws.subscription_id,'\n'
        'Resource group : '+ws.resource_group)

Workspace name : labuser10ml 
Azure region : koreacentral 
Subscription ID : 27db5ec6-d206-4028-b5e1-6004dca5eeef 
Resource group : rg10


In [2]:
# 실험공간을 준비

from azureml.core import Experiment
experiment = Experiment(workspace=ws,name='diabetes-experiment') # Workspace 이름 입력

In [3]:
# 데이터를 준비한다.
# 당뇨병이 걸리는 원인 분석 -> 요인으로 상관관계 분석 = Regression

from azureml.opendatasets import Diabetes
from sklearn.model_selection import train_test_split

X_df = Diabetes.get_tabular_dataset().to_pandas_dataframe().dropna() 
# X값은 변수(요인)
# Tabular : 정적데이터(테이블)
# # dropna: na는 제외

y_df = X_df.pop('Y')
# Y값은 결과(Outcome)
# diabetes Dataframe에서 Y열 제거

X_train , X_test, y_train, y_test = train_test_split(X_df,y_df,test_size=0.2,random_state=66)
# 데이터 나누기

print(X_train)

     AGE  SEX   BMI     BP   S1     S2    S3    S4      S5   S6
440   36    1  30.0   95.0  201  125.2  42.0  4.79  5.1299   85
389   47    2  26.5   70.0  181  104.8  63.0  3.00  4.1897   70
5     23    1  22.6   89.0  139   64.8  61.0  2.00  4.1897   68
289   28    2  31.5   83.0  228  149.4  38.0  6.00  5.3132   83
101   53    2  22.2  113.0  197  115.2  67.0  3.00  4.3041  100
..   ...  ...   ...    ...  ...    ...   ...   ...     ...  ...
122   62    2  33.9  101.0  221  156.4  35.0  6.00  4.9972  103
51    65    2  27.9  103.0  159   96.8  42.0  4.00  4.6151   86
119   53    1  22.0   94.0  175   88.0  59.0  3.00  4.9416   98
316   53    2  27.7   95.0  190  101.8  41.0  5.00  5.4638  101
20    35    1  21.1   82.0  156   87.8  50.0  3.00  4.5109   95

[353 rows x 10 columns]


In [4]:
# 모델 훈련, 로그, 파일 관리
from sklearn.linear_model import Ridge  # Ridge regression : 능형 회귀
from sklearn.metrics import mean_squared_error  # metric = 측정 값 -> mse 방법 이용
from sklearn.externals import joblib # 학습시킨걸 파일로 저장할 수 있음(.pkl)
import math

alphas = [0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,1]
# Ridge 알고리즘 사용 반복문 만들기
# alpha = 기울기 계수의 변화에 따라 능형 회귀 모델의 정확도 측정

for alpha in alphas:

    # 실험의 기록
    run = experiment.start_logging()    # 실험 결과에 따라서 log값을 계속 출력
    run.log('alpha_value',alpha)    # alpha 값에 대한 log 출력 명령

    model = Ridge(alpha=alpha)  # alpha값이 model로 계속 들어가면서 학습됨
    model.fit(X_train,y_train)  # 학습 데이터 입력
    y_pred = model.predict(X_test)  # 예측값에 대한 설정
    rmse = math.sqrt(mean_squared_error(y_test,y_pred))
    # RMSE : Root Mean Squre Error -> 오차를 root로 줄이는 방법.

    run.log('rmse',rmse)    # rmse 값에 대한 log 출력

    print('model_alpha={0}, rmse={1}'.format(alpha, rmse))

    # 모델을 파일로 저장하는 부분
    model_name = 'model_alpha_'+str(alpha) + '.pkl' # alpha가 데이터로 들어가면서 다시 출력용 str으로 변환
    filename = 'outputs/' + model_name

    joblib.dump(value=model, filename=filename)
    # Azrue ML의 job linbrary dump(저장)

    # Azure ML Service에 모델 파일을 업로드 하는 부분
    run.upload_file(name=model_name, path_or_stream=filename)

    run.complete()  # log 완료 명령어

    print(f'{alpha} experiment completed')   # format 지정 f-string

/anaconda/envs/azureml_py38/lib/python3.8/site-packages/sklearn/externals/joblib/__init__.py:15: FutureWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=FutureWarning)


model_alpha=0.1, rmse=56.60520331339142
0.1 expriment completed
model_alpha=0.2, rmse=56.61060264545032
0.2 expriment completed
model_alpha=0.3, rmse=56.616243245483616
0.3 expriment completed
model_alpha=0.4, rmse=56.622107088710145
0.4 expriment completed
model_alpha=0.5, rmse=56.62817734275138
0.5 expriment completed
model_alpha=0.6, rmse=56.63443828302745
0.6 expriment completed
model_alpha=0.7, rmse=56.64087521475942
0.7 expriment completed
model_alpha=0.8, rmse=56.64747440101076
0.8 expriment completed
model_alpha=0.9, rmse=56.654222996253125
0.9 expriment completed
model_alpha=1, rmse=56.66110898499056
1 expriment completed


In [5]:
experiment  # 실험결과 링크

Name,Workspace,Report Page,Docs Page
diabetes-experiment,labuser10ml,Link to Azure Machine Learning studio,Link to Documentation


In [6]:
# Best Model 탐색 후 다운로드
# regression이기 때문에 rmse 값은 작을 수록 좋다 = error 수치가 낮을 수록 연관관계가 높음.

minimum_rmse = None
minimum_rmse_runid = None

for exp in experiment.get_runs():
    run_metrics = exp.get_metrics();    # metrics = rmse
    run_details = exp.get_details();    # details = 실험 ID

    run_rmse = run_metrics['rmse']
    run_id = run_details['runId']

    # 가장 낮은 RMSE 값을 가진 실행 ID를 구하는 부분
    if minimum_rmse is None:    # miminum 초기값은 NONE
        minimum_rmse = run_rmse     # minumum_remse는 run_rmse다
        minimum_rmse_runid = run_id
    else:
        if run_rmse < minimum_rmse: # minumum_rmse 가 none이 아닐경우 결과 rmse를 계속 대입해서 작아지게 됨
            minimum_rmse = run_rmse # run_rmse의 값이 제일 작은 것을 minimum_rmse로 지정
            minimum_rmse_runid = run_id

print('Best_run_id : '+ minimum_rmse_runid)
print('Best_run_id rmse : '+ str(minimum_rmse))

Best_run_id : 50ab9c6c-d5bb-4bfb-acd0-ace0b4226deb
Best_run_id rmse : 56.60520331339142


In [7]:
# Best Model를 가지고 활용하기

from azureml.core import Run
best_run = Run(experiment=experiment, run_id=minimum_rmse_runid)    # 최적의 모델을 실행할 수 있게
print(best_run.get_file_names())

# 파일 다운로드
best_run.download_file(name=str(best_run.get_file_names()[0]))
# [0]은 가장 첫 번째 결과물이 Best model 이기 때문에 입력.

['model_alpha_0.1.pkl', 'outputs/.amlignore', 'outputs/.amlignore.amltmp', 'outputs/model_alpha_0.1.pkl', 'outputs/model_alpha_0.2.pkl', 'outputs/model_alpha_0.3.pkl', 'outputs/model_alpha_0.4.pkl', 'outputs/model_alpha_0.5.pkl', 'outputs/model_alpha_0.6.pkl', 'outputs/model_alpha_0.7.pkl', 'outputs/model_alpha_0.8.pkl', 'outputs/model_alpha_0.9.pkl', 'outputs/model_alpha_1.pkl']


In [8]:
# features/label date 추출하기

import numpy as np
from azureml.core import Dataset

np.savetxt('features.csv', X_train, delimiter=',')
np.savetxt('labels.csv',y_train, delimiter=',')

# DataStore 데이터 저장
datastore = ws.get_default_datastore()  # datastore에 위 data 저장

# Dataupload 데이터 업로드
datastore.upload_files(files=['./features.csv','./labels.csv'],
                        target_path='diabetes-experiment/',
                        overwrite=True)


"datastore.upload_files" is deprecated after version 1.0.69. Please use "FileDatasetFactory.upload_directory" instead. See Dataset API change notice at https://aka.ms/dataset-deprecation.


Uploading an estimated of 2 files
Uploading ./features.csv
Uploaded ./features.csv, 1 files out of an estimated total of 2
Uploading ./labels.csv
Uploaded ./labels.csv, 2 files out of an estimated total of 2
Uploaded 2 files


$AZUREML_DATAREFERENCE_ca13f0743b4b4a59a4cf106a1b2b4e79

In [9]:
# Data 가져오기

feature_dataset = Dataset.Tabular.from_delimited_files(path=[(datastore, 'diabetes-experiment/features.csv')])
label_dataset = Dataset.Tabular.from_delimited_files(path=[(datastore, 'diabetes-experiment/labels.csv')])
# 저장한 파일들을 다시 가져올 수 있다.

In [10]:
#  모델의 등록

import sklearn
from azureml.core import Model
from azureml.core.resource_configuration import ResourceConfiguration

In [13]:
#  모델의 등록 과정

model = Model.register(workspace=ws,
                model_name='diabestes-experiment-model',
                model_path=f'./{str(best_run.get_file_names()[0])}',
                # list 타입이기 때문에 배열 중에 하나를 가져온다. 
                # bestrun은 txt기 때문에 str 문자열로 바꿔준다.
                # 문자열 타입의 정보에 대한 경로를 지정
                # f = foramt -> 함수이기 때문에 formating을 함.
                
                model_framework=Model.Framework.SCIKITLEARN,
                model_framework_version=sklearn.__version__,
                
                sample_input_dataset=feature_dataset,
                sample_output_dataset=label_dataset,
                
                resource_configuration=ResourceConfiguration(cpu=1,memory_in_gb=0.5),
                
                description='Ride regression model to predict diabetes progression',
                tags={'area':'diabetes','type':'regression'}
                )

                # Models 라는 곳에 저장됨
                # 저장이후 endpoint를 활용하면 어디서든 model을 끌어와서 쓸 수 있다.

Registering model diabestes-experiment-model


In [14]:
print('Model name:', model.name)
print('Model version:', model.version)

Model name: diabestes-experiment-model
Model version: 4


In [15]:
# 모델의 배포
service_name = 'diabetes-service'

service = Model.deploy(ws,service_name,[model],overwrite=True)
service.wait_for_deployment(show_output=True)

/tmp/ipykernel_856853/2012493234.py:4: FutureWarning: azureml.core.model:
To leverage new model deployment capabilities, AzureML recommends using CLI/SDK v2 to deploy models as online endpoint, 
please refer to respective documentations 
https://docs.microsoft.com/azure/machine-learning/how-to-deploy-managed-online-endpoints /
https://docs.microsoft.com/azure/machine-learning/how-to-attach-kubernetes-anywhere 
For more information on migration, see https://aka.ms/acimoemigration 
To disable CLI/SDK v1 deprecation warning set AZUREML_LOG_DEPRECATION_WARNING_ENABLED to 'False'
  service = Model.deploy(ws,service_name,[model],overwrite=True)


Tips: You can try get_logs(): https://aka.ms/debugimage#dockerlog or local deployment: https://aka.ms/debugimage#debug-locally to debug if deployment takes longer than 10 minutes.
Running
2023-05-08 02:17:48+00:00 Creating Container Registry if not exists..
2023-05-08 02:27:48+00:00 Registering the environment.
2023-05-08 02:27:49+00:00 Uploading autogenerated assets for no-code-deployment..
2023-05-08 02:27:52+00:00 Building image..
2023-05-08 02:38:05+00:00 Generating deployment configuration.
2023-05-08 02:38:05+00:00 Submitting deployment to compute..
2023-05-08 02:38:11+00:00 Checking the status of deployment diabetes-service..
2023-05-08 02:39:13+00:00 Checking the status of inference endpoint diabetes-service.
Succeeded
ACI service creation operation finished, operation "Succeeded"


In [17]:
# 배포한 모델을 다시 호출 하는 법 (JSON 호출)

import json
 
input_payload = json.dumps({
    'data': X_train[0:2].values.tolist(),
    # X_train 데이터로 분석하기
    'method': 'predict'
})
 
output = service.run(input_payload)
# service : 최적 모델
# 최적 모델로 분석하기
 
print(output)

# 최적 모델을 활용해서 예측값을 반환할 수 있다.

{'predict': [204.9450693706217, 74.4641225933554]}
